In [ ]:
import pandas as pd
from glob import glob
from pypcleaner import convert_excel_time

In [2]:
df = pd.read_hdf('big_multi_index.h5')
scans = pd.read_pickle('scans.p')
imaging_id_codes = pd.read_pickle('imaging_id_codes.p')
id_codes = imaging_id_codes.loc[scans.index,'prefix']
scan_id_df = df.loc[scans.index]
scan_id_df.drop('day_0', axis='columns', level=0, inplace = True)

drop_these = ['bp_three_hr','bp_three_hr_systolic','bp_three_hr_diastolic','bp_three_hr_datetime',
              'bp_seven_hr','bp_seven_hr_systolic','bp_seven_hr_diastolic','bp_seven_hr_datetime',
              'bp_other_hr','bp_other_hr_systolic','bp_other_hr_diastolic','bp_other_hr_datetime',
              'hrt_seven_hr','hrt_seven_hr_rate','hrt_seven_hr_datetime','hrt_other_hr',
              'hrt_other_hr_rate','hrt_other_hr_datetime','surgical_intervention',
              'surgical_intervention_detail','invasive_intervention','invasive_intervention_detail',
              'imaging_intervention','imaging_intervention_detail',
              'other_intervention','other_intervention_detail','medication_given','medication_comment']

scan_id_df.drop([('day_baseline','recent_medical_history',c) for c in drop_these], axis = 'columns',inplace = True)

scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')] = \
scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')].apply(convert_excel_time).dt.round('5min')

In [15]:
import itertools

In [17]:
itertools.combinations(['a','b'])

TypeError: Required argument 'r' (pos 2) not found

In [3]:
remove_tests = \
[('day_baseline','physical_examination'), 
 ('day_baseline', 'past_medical_history'), 
 ('day_baseline', 'CCMI'), 
 ('day_baseline', 'further_stroke_event_part'), 
 ('day_baseline', 'bloods'), 
 ('day_baseline', 'laboratory_tests'), 
 ('day_baseline', 'radiological_scans'), 
 ('day_baseline', 'concomitant_medications'), 
 ('day_baseline', 'pre-stroke_mRS'),
 ('day_7', 'bloods') , 
 ('day_7', 'history_of_depression'), 
 ('day_7', 'physical_risk_factors'), 
 ('day_7', 'diet_q'),
 ('day_7', 'laboratory_tests'), 
 ('day_7', 'concomitant_medications'), 
 ('day_7', 'concomitant_medications_plus'), 
 ('day_90', 'bloods'), 
 ('day_90', 'physical_examination'), 
 ('day_90', 'physical_risk_factors'), 
 ('day_90', 'diet_q'), 
 ('day_90', 'laboratory_tests'), 
 ('day_90', 'further_stroke_event_part'), 
 ('day_90', 'history_of_depression'), 
 ('day_90', 'concomitant_medications'), 
 ('day_365', 'bloods'), 
 ('day_365', 'physical_examination'), 
 ('day_365', 'physical_risk_factors'), 
 ('day_365', 'diet_q'), 
 ('day_365', 'laboratory_tests'), 
 ('day_365', 'further_stroke_event_part'), 
 ('day_365', 'history_of_depression'), 
 ('day_365', 'concomitant_medications'), 
 ('day_1', 'vital_signs'), 
 ('day_1', 'bloods'), 
 ('day_1', 'laboratory_tests'), 
 ('day_1', 'serious_adverse_event')]

scan_id_df.drop(remove_tests, axis = 'columns',inplace = True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [5]:
demographics = pd.read_pickle('base_df_data.p')


stroke_onset = demographics.loc[scans.index,['identity_stroke_onset']]
multi_NIHSS = scan_id_df[('day_baseline','NIHSS_multiple')]
multi_with_stroke_onset = stroke_onset.join(multi_NIHSS)

for c in ['nihss_three_hr_datetime', 'nihss_seven_hr_datetime', 
          'nihss_eighteen_hr_datetime', 'nihss_other_hr_datetime']:
    hrs = multi_with_stroke_onset[c] - multi_with_stroke_onset['identity_stroke_onset']
    multi_with_stroke_onset[c] = hrs.dt.round('5min')
    
scan_id_df[('day_baseline','NIHSS_multiple')] = multi_with_stroke_onset

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


In [6]:
bit_1_cols = demographics.loc[:,'identity_gender':'identity_age_at_stroke_in_years'].columns
bit_2_cols = demographics.loc[:,'demographics_ancestry_1':].columns
useful_columns = bit_1_cols.append(bit_2_cols)
demographics = demographics.loc[scans.index,useful_columns]

In [7]:
demographics['identity_gender'] = demographics['identity_gender'].map({1:'female',0:'male'})

In [8]:
id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,'prefix']
infarct_type = scan_id_df[('day_1','infarct_type')]
scan_id_df[('day_90','lesion_location_confirmed')].join([id_codes,infarct_type])

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,lesion location confirmed by imaging at 3 months (BC report),prefix,is_ich,date_time,symptomatic,loc_frontal,loc_temporal,loc_parietal,loc_occipital,loc_subcortical,loc_brainstem,loc_cerebellar,loc_subdural,loc_subarachnoid,treatment,treatment_spec
119,right,D_PP_P01-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
123,right,D_PP_P02-01-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
124,left,D_PP_P12-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
126,right,D_PP_P01-10-NP,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
127,right,D_PP_P12-02-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
129,right,D_PP_P01-07-GS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
132,right,D_PP_P02-02-SH,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
133,left pons,D_PP_P01-11-GW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
136,right,D_PP_P15-01-EM,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
137,right,D_PP_P15-02-DK,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [46]:
print(scan_id_df.xs('TUGT_average',level=1,axis = 'columns').join(scan_id_df.xs('TUGT',level=1,axis = 'columns')).to_html().replace('\n',''))

<table border="1" class="dataframe">  <thead>    <tr>      <th></th>      <th colspan="5" halign="left">day_90</th>      <th colspan="4" halign="left">day_365</th>    </tr>    <tr>      <th></th>      <th>time1</th>      <th>time2</th>      <th>time3</th>      <th>TUGT_average</th>      <th>time_taken</th>      <th>time1</th>      <th>time2</th>      <th>time3</th>      <th>average</th>    </tr>  </thead>  <tbody>    <tr>      <th>119</th>      <td>10.0</td>      <td>10.0</td>      <td>10.0</td>      <td>10.0</td>      <td>213.0</td>      <td>NaN</td>      <td>NaN</td>      <td>NaN</td>      <td>NaN</td>    </tr>    <tr>      <th>123</th>      <td>NaN</td>      <td>NaN</td>      <td>NaN</td>      <td>NaN</td>      <td>300.0</td>      <td>30.0</td>      <td>30.0</td>      <td>30.0</td>      <td>30.0</td>    </tr>    <tr>      <th>124</th>      <td>6.0</td>      <td>7.0</td>      <td>7.0</td>      <td>6.7</td>      <td>237.0</td>      <td>8.0</td>      <td>8.0</td>      <td>7.0</td>     

In [ ]:
pd.DataFrame.to_html()

In [27]:
scan_id_df[('day_90','TUGT')].head()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,time_taken
119,213.0
123,300.0
124,237.0
126,300.0
127,47.0


In [29]:
scan_id_df[('day_90','TUGT_average')].head()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,time1,time2,time3,TUGT_average
119,10.0,10.0,10.0,10.0
123,NaN,NaN,NaN,NaN
124,6.0,7.0,7.0,6.7
126,11.0,9.0,10.0,10.0
127,6.0,5.0,5.0,4.7


In [35]:
scan_id_df[('day_365')].columns.tolist()

[('mRS', 'score'),
 ('mRS', 'resp'),
 ('mRS', 'resp_other'),
 ('mRS', 'date'),
 ('mRS', 'valid'),
 ('mRS', 'reason'),
 ('NIHSS', 'loc'),
 ('NIHSS', 'loc_quest'),
 ('NIHSS', 'loc_command'),
 ('NIHSS', 'best_gaze'),
 ('NIHSS', 'vis_field_test'),
 ('NIHSS', 'facial_palsy'),
 ('NIHSS', 'motor_arm_l'),
 ('NIHSS', 'motor_arm_r'),
 ('NIHSS', 'motor_leg_l'),
 ('NIHSS', 'motor_leg_r'),
 ('NIHSS', 'limb_ataxia'),
 ('NIHSS', 'sensory'),
 ('NIHSS', 'best_lang'),
 ('NIHSS', 'dysarthria'),
 ('NIHSS', 'extinction'),
 ('NIHSS', 'score'),
 ('NIHSS', 'valid'),
 ('NIHSS', 'reason'),
 ('BI', 'resp.1'),
 ('BI', 'resp_other.1'),
 ('BI', 'feeding'),
 ('BI', 'bathing'),
 ('BI', 'grooming'),
 ('BI', 'dressing'),
 ('BI', 'bowels'),
 ('BI', 'bladder'),
 ('BI', 'toilet_use'),
 ('BI', 'transfers'),
 ('BI', 'mobility'),
 ('BI', 'stairs'),
 ('BI', 'score'),
 ('SIS', 'resp.2'),
 ('SIS', 'resp_other.2'),
 ('SIS', 'phy_arm'),
 ('SIS', 'phy_hand'),
 ('SIS', 'phy_leg'),
 ('SIS', 'phy_foot'),
 ('SIS', 'mem_told'),
 ('SIS'

In [42]:
scan_id_df[('day_90','TUGT')]

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,time_taken
119,213.0
123,300.0
124,237.0
126,300.0
127,47.0
129,300.0
132,193.0
133,51.0
136,300.0
137,110.0


In [61]:
scan_id_df[('day_90','TUGT_average','correct_average')] = scan_id_df['day_90']['TUGT_average'][['time1', 'time2', 'time3']].mean(axis = 'columns').round(1)

In [64]:
scan_id_df['day_90']['TUGT_average'].head()

,time1,time2,time3,TUGT_average,correct_average
119,10.0,10.0,10.0,10.0,10.0
123,NaN,NaN,NaN,NaN,NaN
124,6.0,7.0,7.0,6.7,6.7
126,11.0,9.0,10.0,10.0,10.0
127,6.0,5.0,5.0,4.7,5.3


In [69]:
scan_id_df[('day_365','TUGT','correct_average')] = scan_id_df['day_365']['TUGT'][['time1', 'time2', 'time3']].mean(axis='columns').round(1)

In [71]:
scan_id_df['day_365']['TUGT'].head()

,time1,time2,time3,average,correct_average
119,NaN,NaN,NaN,NaN,NaN
123,30.0,30.0,30.0,30.0,30.0
124,8.0,8.0,7.0,7.3,7.7
126,9.0,9.0,10.0,8.7,9.3
127,6.0,6.0,6.0,5.7,6.0


In [75]:
scan_id_df['day_90']['TUGT'].head()

,time_taken
119,213.0
123,300.0
124,237.0
126,300.0
127,47.0


### Do it!

delete the individual columns and keep the calculated average